## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [29]:
import numpy as np
import pandas as pd

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 69

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯ'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [30]:
train_companies = open('kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

Найдем самое длинное слово (для нормализации), плюс добавим пару функций для генерации фич

In [31]:
import nltk
from nltk import word_tokenize

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

def find_max_str(companies):
    max_str = ''
    for company in companies:
        for word in word_tokenize(company):
            if(has_letters(word)):
                max_str = word if len(max_str) < len(word) else max_str 
    return max_str

def find_index_of_substring(string, companies):
    for i in range(len(companies)):
        if(companies[i].find(string) != -1):
            return i

В данной строке артефакт, подробнее в 'Testing.ipynb'

In [32]:
del train_companies[471970]

In [33]:
def extract_features(company_name, test=False):
    
    #Длина слова, количество гласных/согласных, положение в слове и есть ли в слове что-то помимо букв
    num_of_features = 5
    
    #3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            features[index][0] = index/size
            features[index][1] = len(word)/MAX_WORD
            features[index][2] = count_consonants(word)/len(word)
            features[index][3] = count_vowels(word)/len(word)
            features[index][4] = int(has_not_letters(word))
            if(word[0].isupper()):
                if(word[-1].isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return features, labels
    else:
        return features

In [36]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features = np.concatenate((features, curr_features))
            labels = np.concatenate((labels, curr_labels))
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [37]:
def transform(company_name, labels):
    index = 0
    for word in word_tokenize(company_name):
        if(has_letters(word)):
            if(labels[index] == 0):
                print(word.lower())
            elif(labels[index] == 1):
                print(word[0] + word[1:].lower())
            else:
                print(word)
            index += 1

In [38]:
X_train, y_train = generate_dataset(train_companies[:50000])

In [39]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier()
clf.fit(X_train, y_train)

Learning rate set to 0.106383
0:	learn: 0.9961313	total: 130ms	remaining: 2m 9s
1:	learn: 0.9180479	total: 217ms	remaining: 1m 48s
2:	learn: 0.8554086	total: 299ms	remaining: 1m 39s
3:	learn: 0.8016490	total: 380ms	remaining: 1m 34s
4:	learn: 0.7587590	total: 464ms	remaining: 1m 32s
5:	learn: 0.7229263	total: 546ms	remaining: 1m 30s
6:	learn: 0.6915697	total: 631ms	remaining: 1m 29s
7:	learn: 0.6642231	total: 718ms	remaining: 1m 29s
8:	learn: 0.6406460	total: 801ms	remaining: 1m 28s
9:	learn: 0.6198662	total: 880ms	remaining: 1m 27s
10:	learn: 0.6020778	total: 964ms	remaining: 1m 26s
11:	learn: 0.5860692	total: 1.05s	remaining: 1m 26s
12:	learn: 0.5720652	total: 1.12s	remaining: 1m 25s
13:	learn: 0.5598649	total: 1.21s	remaining: 1m 25s
14:	learn: 0.5476841	total: 1.29s	remaining: 1m 24s
15:	learn: 0.5373765	total: 1.37s	remaining: 1m 24s
16:	learn: 0.5278632	total: 1.46s	remaining: 1m 24s
17:	learn: 0.5197207	total: 1.53s	remaining: 1m 23s
18:	learn: 0.5121160	total: 1.61s	remaining: 

157:	learn: 0.3804425	total: 13s	remaining: 1m 9s
158:	learn: 0.3802752	total: 13.1s	remaining: 1m 9s
159:	learn: 0.3800087	total: 13.1s	remaining: 1m 8s
160:	learn: 0.3797833	total: 13.2s	remaining: 1m 8s
161:	learn: 0.3796399	total: 13.3s	remaining: 1m 8s
162:	learn: 0.3794118	total: 13.4s	remaining: 1m 8s
163:	learn: 0.3792681	total: 13.5s	remaining: 1m 8s
164:	learn: 0.3790109	total: 13.6s	remaining: 1m 8s
165:	learn: 0.3788244	total: 13.6s	remaining: 1m 8s
166:	learn: 0.3785528	total: 13.7s	remaining: 1m 8s
167:	learn: 0.3783369	total: 13.8s	remaining: 1m 8s
168:	learn: 0.3781983	total: 13.9s	remaining: 1m 8s
169:	learn: 0.3779938	total: 14s	remaining: 1m 8s
170:	learn: 0.3777251	total: 14s	remaining: 1m 8s
171:	learn: 0.3775985	total: 14.1s	remaining: 1m 7s
172:	learn: 0.3772568	total: 14.2s	remaining: 1m 7s
173:	learn: 0.3770924	total: 14.3s	remaining: 1m 7s
174:	learn: 0.3768227	total: 14.4s	remaining: 1m 7s
175:	learn: 0.3766013	total: 14.5s	remaining: 1m 7s
176:	learn: 0.3765

319:	learn: 0.3606119	total: 26.3s	remaining: 55.8s
320:	learn: 0.3605584	total: 26.3s	remaining: 55.7s
321:	learn: 0.3604791	total: 26.4s	remaining: 55.6s
322:	learn: 0.3603764	total: 26.5s	remaining: 55.6s
323:	learn: 0.3603250	total: 26.6s	remaining: 55.5s
324:	learn: 0.3602368	total: 26.7s	remaining: 55.4s
325:	learn: 0.3602103	total: 26.8s	remaining: 55.3s
326:	learn: 0.3601103	total: 26.8s	remaining: 55.3s
327:	learn: 0.3600429	total: 26.9s	remaining: 55.2s
328:	learn: 0.3599848	total: 27s	remaining: 55.1s
329:	learn: 0.3599373	total: 27.1s	remaining: 55s
330:	learn: 0.3598743	total: 27.2s	remaining: 54.9s
331:	learn: 0.3598016	total: 27.3s	remaining: 54.8s
332:	learn: 0.3597250	total: 27.3s	remaining: 54.8s
333:	learn: 0.3596019	total: 27.4s	remaining: 54.7s
334:	learn: 0.3595358	total: 27.5s	remaining: 54.6s
335:	learn: 0.3595055	total: 27.6s	remaining: 54.5s
336:	learn: 0.3594633	total: 27.7s	remaining: 54.4s
337:	learn: 0.3594163	total: 27.7s	remaining: 54.3s
338:	learn: 0.35

478:	learn: 0.3520437	total: 39.3s	remaining: 42.8s
479:	learn: 0.3519744	total: 39.4s	remaining: 42.7s
480:	learn: 0.3519170	total: 39.5s	remaining: 42.6s
481:	learn: 0.3518702	total: 39.6s	remaining: 42.5s
482:	learn: 0.3518379	total: 39.6s	remaining: 42.4s
483:	learn: 0.3518013	total: 39.7s	remaining: 42.3s
484:	learn: 0.3517695	total: 39.8s	remaining: 42.3s
485:	learn: 0.3516655	total: 39.9s	remaining: 42.2s
486:	learn: 0.3515991	total: 40s	remaining: 42.1s
487:	learn: 0.3515708	total: 40.1s	remaining: 42s
488:	learn: 0.3515206	total: 40.1s	remaining: 41.9s
489:	learn: 0.3514836	total: 40.2s	remaining: 41.9s
490:	learn: 0.3514266	total: 40.3s	remaining: 41.8s
491:	learn: 0.3513788	total: 40.4s	remaining: 41.7s
492:	learn: 0.3513633	total: 40.5s	remaining: 41.6s
493:	learn: 0.3513247	total: 40.5s	remaining: 41.5s
494:	learn: 0.3512863	total: 40.6s	remaining: 41.5s
495:	learn: 0.3512558	total: 40.7s	remaining: 41.4s
496:	learn: 0.3512455	total: 40.8s	remaining: 41.3s
497:	learn: 0.35

638:	learn: 0.3466806	total: 53.3s	remaining: 30.1s
639:	learn: 0.3466725	total: 53.4s	remaining: 30s
640:	learn: 0.3466414	total: 53.5s	remaining: 30s
641:	learn: 0.3466192	total: 53.6s	remaining: 29.9s
642:	learn: 0.3465609	total: 53.8s	remaining: 29.8s
643:	learn: 0.3465287	total: 53.9s	remaining: 29.8s
644:	learn: 0.3464930	total: 54s	remaining: 29.7s
645:	learn: 0.3464764	total: 54.1s	remaining: 29.7s
646:	learn: 0.3464488	total: 54.2s	remaining: 29.6s
647:	learn: 0.3464175	total: 54.3s	remaining: 29.5s
648:	learn: 0.3464031	total: 54.4s	remaining: 29.4s
649:	learn: 0.3463741	total: 54.5s	remaining: 29.3s
650:	learn: 0.3463394	total: 54.6s	remaining: 29.3s
651:	learn: 0.3463056	total: 54.7s	remaining: 29.2s
652:	learn: 0.3462859	total: 54.8s	remaining: 29.1s
653:	learn: 0.3462653	total: 54.9s	remaining: 29s
654:	learn: 0.3462528	total: 54.9s	remaining: 28.9s
655:	learn: 0.3462309	total: 55s	remaining: 28.9s
656:	learn: 0.3461968	total: 55.1s	remaining: 28.8s
657:	learn: 0.3461419	

800:	learn: 0.3429302	total: 1m 9s	remaining: 17.2s
801:	learn: 0.3429073	total: 1m 9s	remaining: 17.1s
802:	learn: 0.3428981	total: 1m 9s	remaining: 17.1s
803:	learn: 0.3428839	total: 1m 9s	remaining: 17s
804:	learn: 0.3428502	total: 1m 9s	remaining: 16.9s
805:	learn: 0.3428277	total: 1m 9s	remaining: 16.8s
806:	learn: 0.3428189	total: 1m 9s	remaining: 16.7s
807:	learn: 0.3428088	total: 1m 9s	remaining: 16.6s
808:	learn: 0.3427968	total: 1m 10s	remaining: 16.5s
809:	learn: 0.3427783	total: 1m 10s	remaining: 16.5s
810:	learn: 0.3427660	total: 1m 10s	remaining: 16.4s
811:	learn: 0.3427569	total: 1m 10s	remaining: 16.3s
812:	learn: 0.3427432	total: 1m 10s	remaining: 16.2s
813:	learn: 0.3427109	total: 1m 10s	remaining: 16.1s
814:	learn: 0.3426975	total: 1m 10s	remaining: 16s
815:	learn: 0.3426926	total: 1m 10s	remaining: 16s
816:	learn: 0.3426823	total: 1m 10s	remaining: 15.9s
817:	learn: 0.3426725	total: 1m 10s	remaining: 15.8s
818:	learn: 0.3426499	total: 1m 11s	remaining: 15.7s
819:	le

958:	learn: 0.3402989	total: 1m 22s	remaining: 3.54s
959:	learn: 0.3402874	total: 1m 22s	remaining: 3.46s
960:	learn: 0.3402716	total: 1m 23s	remaining: 3.37s
961:	learn: 0.3402512	total: 1m 23s	remaining: 3.28s
962:	learn: 0.3402405	total: 1m 23s	remaining: 3.2s
963:	learn: 0.3402193	total: 1m 23s	remaining: 3.11s
964:	learn: 0.3402020	total: 1m 23s	remaining: 3.02s
965:	learn: 0.3401891	total: 1m 23s	remaining: 2.94s
966:	learn: 0.3401733	total: 1m 23s	remaining: 2.85s
967:	learn: 0.3401565	total: 1m 23s	remaining: 2.77s
968:	learn: 0.3401484	total: 1m 23s	remaining: 2.68s
969:	learn: 0.3401209	total: 1m 23s	remaining: 2.59s
970:	learn: 0.3400138	total: 1m 23s	remaining: 2.51s
971:	learn: 0.3399963	total: 1m 24s	remaining: 2.42s
972:	learn: 0.3399847	total: 1m 24s	remaining: 2.33s
973:	learn: 0.3399753	total: 1m 24s	remaining: 2.25s
974:	learn: 0.3399175	total: 1m 24s	remaining: 2.16s
975:	learn: 0.3399145	total: 1m 24s	remaining: 2.07s
976:	learn: 0.3399090	total: 1m 24s	remaining: 

In [40]:
company = 'ООО РК "Восток"'
extract_features(company, test=True)

array([[0.        , 0.04347826, 0.        , 1.        , 0.        ],
       [0.33333333, 0.02898551, 1.        , 0.        , 0.        ],
       [0.66666667, 0.08695652, 0.66666667, 0.33333333, 0.        ]])

In [42]:
transform(company, clf.predict(extract_features(company, test=True)))

ООО
РК
Восток


In [44]:
train_companies[2000:2050]

['Общество с ограниченной ответственностью "Зарем- Маркет"',
 'ООО "Зарем-Маркет"',
 'Общество с ограниченной ответственностью "АИР"',
 'Государственное учреждение здравоохранения "Адыгейская республиканская станция переливания крови"',
 'Государственное бюджетное учреждение здравоохранения Республики Адыгея "Адыгейская республиканская станция переливания крови"',
 'Общество с ограниченной ответственностью "Альт"',
 'Общество с ограниченной ответственностью "АгроХимСервис"',
 'Управление Судебного департамента в Республике Адыгея',
 'Общество с ограниченной ответственностью "Крестьянское хозяйство "Спикор"',
 'Территориальная избирательная комиссия города Майкопа',
 'Благотворительный фонд "Диамонт"',
 'Управление ГОЧС г. Майкопа',
 'Управление по делам гражданской обороны и чрезвычайным ситуациям администрации муниципального образования "Город Майкоп"',
 'Управление по чрезвычайным ситуациям Администрации муниципального образования "Город Майкоп"',
 'Управление ЧС г. Майкопа',
 'Общес